In [23]:
import requests
import csv
import pandas as pd

In [24]:
# Creating the empty dataset with the column names, take any municipi
def create_dataset(municipi_code):
    concepteids = ['f69', 'f72', 'f73', 'f74', 'f183', 'f184', 'f263', 'f380', 'f381', 'f98', 'f99', 'f100', 'f101', 'f107', 'f109', 'f111', 'f112', 'f113', 'f115', 'f221', 'f222', 'f223', 'f60', 'f242', 'f250', 'f120', 'f122', 'f126', 'f128', 'f131', 'f190', 'f191', 'f193', 'f385']

    url = f"https://www.idescat.cat/emex/?id={municipi_code}&f=ssv"
    df = pd.read_csv(url, skiprows=5, delimiter=";")
    df = df[df["concepteid"].isin(concepteids)]
    df["unitat"] = df["unitat"].fillna('').astype(str)
    df['concepte'] = df.agg('{0[taula]} - {0[concepte]} ({0[temps]}) {0[unitat]}'.format, axis=1)
    df.drop(['tema', 'temaid', 'taulaid', 'concepteid', 'font', 'valorcom', 'valorcat', 'taula', 'temps', 'unitat'], axis=1, inplace=True)
    cols = ["muncipi"] + list(df["concepte"])
    values = [municipi_code] + [x.replace(',', '.') for x in list(df["valormun"])]
    return pd.DataFrame(columns=cols, data=[values])

In [25]:
codis_municipis = ['430120', '439076', '430430', '430477', '430509', '430957', '430976', '431009', '431037', '431093', '431114', '431227', '431264', '431310', '431359', '439057', '431444', '431482', '431536', '431649', '431711', '431665']
df = create_dataset(codis_municipis[0])
for municipi_code in codis_municipis[1:]:
    try:
        dfnew = create_dataset(municipi_code)
        df = pd.concat([df, dfnew])
    except:
        print(municipi_code)


In [26]:
df = df.apply(pd.to_numeric)
# Save into excel file
df.to_excel("output/poblacio_habitatges.xlsx", index=False)